# Multi-Sensor Atmospheric Correction in Google Earth Engine

**Description:** This script allows to do atmospheric correction on only individual images of Sentinel-2 and Landsat sensors, especifically for images over coastal or oceanic areas. These settings can be modified from the *parameters.py* module to work with images over inland areas (See line 36 in that module). The script does AC automatically, the user only needs to provide the right **mission**, **imageID**, and specific **assetID** to export the processed image to your EE Assets.<br/>
More sensors can be added by modifying the *mission_specifics.py* and *parameters.py* modules to properly work with the available collections in GEE and [Py6S](https://github.com/robintw/Py6S/blob/master/Py6S/Params/wavelength.py).<br/>

Script modified from https://github.com/samsammurphy/gee-atmcorr-S2<br/>
By Luis Lizcano-Sandoval<br/>
College of Marine Science, University of South Florida<br/>
09/25/2020<br/>

### Import modules and initialize Earth Engine



In [10]:
import ee
from Py6S import *
import datetime
import math
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from atmospheric import Atmospheric
import mission_specifics as mn
from parameters import BOA
import timeit

ee.Initialize()

### Earth Engine Collections
Set the collection of interest using one this specific categories:


In [11]:
mission = 'Sentinel2'
#mission = 'Landsat8'
#mission = 'Landsat7'
#mission = 'Landsat5'

### Image ID
Paste the ID of your target image.

In [12]:
imageID = '20190822T155829_20190822T161509_T17RLM' #Sentinel-2
#imageID = 'LC08_006038_20141002' #Landsat8
#imageID = 'LE07_006038_20020315' #Landsat7
#imageID = 'LT05_017040_20061009' #Landsat5

### Load image
Get the respective scene from the collection

In [13]:
# Load image
image = ee.Image(mn.eeCollection(mission) +'/'+ imageID)
print('Image: ', image.getInfo()['properties']['system:index'])

# Date
dateString = datetime.datetime.utcfromtimestamp(image.get('system:time_start').getInfo()/1000).strftime("%Y-%m-%d")
print('Date: ',dateString)

# If working with S-2, then identify whether the sensor is Sentinel-2A or Sentinel-2B, and rename the variable 'mission'
if 'Sentinel2' == mission:
    mission = str(image.getInfo()['properties']['SPACECRAFT_NAME'])
print('Mission: ', mission)

Image:  20190822T155829_20190822T161509_T17RLM
Date:  2019-08-22
Mission:  Sentinel-2B


### Atmospheric Correction
Available bands available for atmospheric correction for each sensor, according to the [Py6S module](https://github.com/robintw/Py6S/blob/master/Py6S/Params/wavelength.py): <br/>
* **Sentinel2:** ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'],<br/>
* **Landsat8:** ['B1','B2','B3','B4','B5','B6','B7','B8','B9'],<br/>
* **Landsat7:** ['B1','B2','B3','B4','B5','B7'],<br/>
* **Landsat5:** ['B1','B2','B3','B4','B5','B7'],<br/>
* **Landsat4:** ['B1','B2','B3','B4','B5','B7']<br/>

The respective cloud mask band will be preserved after the correction:<br/>
* **Sentinel2:** 'QA60',<br/>
* **Landsat sensors:** 'BQA'

In [14]:
%%time
print('Processing...')
# Surface reflectance (You can add more bands as desired, according to the above info)
b1 = BOA(mission, image, 'B1')
b2 = BOA(mission, image, 'B2')
b3 = BOA(mission, image, 'B3')
b4 = BOA(mission, image, 'B4')
b5 = BOA(mission, image, 'B5')
b8 = BOA(mission, image, 'B8')
b11 = BOA(mission, image, 'B11')
b12 = BOA(mission, image, 'B12')

# Cloud mask band
qa = []
if 'Sentinel' in mission:
    qa = image.select('QA60')#For Sentinel
else:
    qa = image.select('BQA') #For Landsat

Wall time: 19.4 s


**NOTE:** The function *positive* will convert any negative value to 0.0001 in all bands. For Sentinel-2, the bands B1,B2,B3,B4 are more susceptible to present negative values in very dark/coastal areas. I have compared those areas using Sentinel-2 L2A images and it seems they do the same: dark areas showing default minimum valid pixel values of 0.0001. B8 might show negative reflectances on coastal and oceanic areas, but not on cloudy pixels, so it does not affect cloud masking procedures. In my case, bands B8, B11, B12 are only used for masking clouds. If you need to use these bands for other purposes just check that the areas of negative values are not large, otherwise I would not recommend to use them (it is up to you). The negative reflectances might be due to overestimations of aerosols at sea-level in coastal areas and suspendend particles in water. Landsat sensors may show a similar behaviour. Band B10 should not be provided as a surface reflectance output, because it does not provide information on the surface but on the cirrus clouds [[Main-Knorn et al. 2017]](https://www.researchgate.net/publication/320231869_Sen2Cor_for_Sentinel-2). 

In [15]:
## Function to convert negative reflectances to 0.0001
def positive(band):
    b = band.gt(0)
    b_mask = band.mask(b)
    return ee.Image(b_mask).unmask(0.0001)

## Bands to convert to positive.
b1m = positive(b1)
b2m = positive(b2)
b3m = positive(b3)
b4m = positive(b4)
b5m = positive(b5)
b8m = positive(b8)
b11m = positive(b11)
b12m = positive(b12)

## Getting all the bands together
output = b1m.addBands(b2m).addBands(b3m).addBands(b4m).addBands(b5m)\
        .addBands(b8m).addBands(b11m).addBands(b12m).addBands(qa)

## Copy properties from the original image
output = output.set(image.toDictionary(image.propertyNames()))

### Display results

In [7]:
from IPython.display import display, Image

region = image.geometry().buffer(5000).bounds().getInfo()['coordinates']

# RGB Bands
channels = []
if 'Sentinel' in mission or 'Landsat8' == mission:
    channels = ['B4','B3','B2'] #For Sentinel & Landsat8
else:
    channels = ['B3','B2','B1'] #For Landsat7-5-4

# Display images:
original = Image(url=mn.TOA(image,mission).select(channels).getThumbUrl({
    'dimensions': '1000x1000',
    'min':0,
    'max':0.25
    }))

corrected = Image(url=output.select(channels).getThumbUrl({
    'dimensions': '1000x1000',
    'min':0,
    'max':0.25,
    'gamma':1.5
    }))

display(original, corrected)

### Export to Asset

NOTE: 
* Be aware that each band will be resampled at the scale used to export the image. This will impact the size of the exported file.
* A Sentinel-2 image with 8 bands at 10m res each can occupy ~1.5 gb.
* A Sentinel-2 image with 8 bands can take ~4-8 min to ingest.

In [16]:
# Set the scale properly
scale = []
sat = []
tile = []
if 'Sentinel' in mission:
    sat = 'Sentinel'
    scale = 10 #For Sentinel
    tile = image.getInfo()['properties']['MGRS_TILE']
else:
    sat = 'Landsat'
    scale = 30 #For Landsat 
    tile = (image.getInfo()['properties']['WRS_PATH'])+(image.getInfo()['properties']['WRS_ROW'])
    
# set some properties for export
output = output.set({'satellite': mission,
               'tile_id': tile,
               'file_id': imageID,                                               
               'date': dateString,
               'generator': 'Lizcano-Sandoval',
                    })

# define YOUR assetID. (This do not create folders, you need to create them manually)
assetID = 'users/lizcanosandoval/BOA/'+sat+'/'+'FL_19/' ##This goes to an ImageCollection folder
fileName = tile+'_'+imageID+'_BOA'
path = assetID + fileName

In [17]:
## export
export = ee.batch.Export.image.toAsset(\
        image = output,                                                    
        description = 'S2_BOA_'+imageID,
        assetId = path,
        region = image.geometry().buffer(10),                                      
        maxPixels = 1e9,
        scale = scale)

# # uncomment to run the export
export.start() 